#### Alfonso Villegas Narvaez
Kaggle Competition (Only 3 hours to create a model and a submission)

MBD - Section 2

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV

In [2]:
train_m = pd.read_csv('./data/train_main.csv')
train_g = pd.read_csv('./data/train_geo.csv')
train_w = pd.read_csv('./data/train_weather.csv')

In [3]:
test_m = pd.read_csv('./data/test_main.csv')
test_g = pd.read_csv('./data/test_geo.csv')
test_w = pd.read_csv('./data/test_weather.csv')

In [4]:
holidays = pd.read_csv('./data/holidays.csv')
sample_submission = pd.read_csv('./data/sample_submission.csv')

In [5]:
train_m
# id = id unique to each delivery
# starting_time = packages loaded and driver starts the engine
# delivery_time = target variable, time in seconds

,id,starting_time,model_van,num_packages,delivery_time
0,1,2016/04/14 20:53:00,model_A,3,448
1,2,2016/05/30 23:12:00,model_B,1,784
2,3,2016/03/03 11:54:00,model_B,1,844
3,4,2016/05/02 12:34:00,model_A,1,2076
4,5,2016/03/02 15:53:00,model_A,1,1289
...,...,...,...,...,...
8489,8490,2016/05/02 08:27:00,model_A,6,689
8490,8491,2016/06/05 07:56:00,model_B,3,538
8491,8492,2016/03/10 12:49:00,model_B,1,3093
8492,8493,2016/05/02 20:52:00,model_B,1,289


In [6]:
train_g

,id,town,start_Longitude,start_Latitude,end_Longitude,end_Latitude
0,7195,Hawkins,-73.957771,40.717770,-73.926498,40.705521
1,4844,Hawkins,-73.973984,40.750404,-73.872925,40.774326
2,2402,Hawkins,-73.977150,40.751656,-73.981308,40.782711
3,6996,Hawkins,-73.984489,40.736862,-73.960564,40.769562
4,4207,Hawkins,-73.976616,40.790688,-73.974281,40.783520
...,...,...,...,...,...,...
8489,2338,Hawkins,-73.991417,40.749920,-73.996330,40.726730
8490,1147,Hawkins,-73.986427,40.767189,-73.983398,40.760872
8491,5383,Hawkins,-73.960289,40.807659,-73.967529,40.762440
8492,5067,Hawkins,-73.974365,40.760712,-73.956078,40.763664


In [7]:
train_w

,id,wea_conditions,wea_temp,wea_dptemp,wea_humidity,wea_windspeed,wea_winddir,wea_visibility,wea_pressure,wea_precipitation,wea_fog,wea_rain,wea_snow
0,7322,Mostly_Cloudy,22.8,11.7,49.0,7.4,0,16.1,29.9,NaN,No,No,No
1,5677,Clear,12.8,-2.8,34.0,0.0,0,16.1,29.9,NaN,No,No,No
2,5269,Clear,16.7,-6.7,20.0,5.6,0,16.1,30.3,NaN,No,No,No
3,7334,Overcast,9.4,7.2,86.0,13.0,70,16.1,29.8,0.8,No,No,No
4,3356,Light_Rain,11.7,10.6,93.0,5.6,0,4.0,29.9,1.8,No,Y,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8489,4403,Haze,11.7,8.9,83.0,NaN,0,9.7,29.7,0.3,No,No,No
8490,1073,Clear,10.0,2.8,61.0,25.9,60,16.1,30.1,NaN,No,No,No
8491,2026,Clear,-6.1,-11.1,68.0,7.4,0,16.1,29.9,NaN,No,No,No
8492,7646,Overcast,11.7,-3.9,34.0,16.7,300,16.1,29.8,NaN,No,No,No


In [8]:
# Merging Training Dataframes on id
train_g1 = pd.merge(train_m, train_g, on = 'id')
train_g1

,id,starting_time,model_van,num_packages,delivery_time,town,start_Longitude,start_Latitude,end_Longitude,end_Latitude
0,1,2016/04/14 20:53:00,model_A,3,448,Hawkins,-73.950912,40.825974,-73.944740,40.814163
1,2,2016/05/30 23:12:00,model_B,1,784,Hawkins,-73.985680,40.760487,-73.942482,40.786232
2,3,2016/03/03 11:54:00,model_B,1,844,Hawkins,-73.985420,40.758389,-74.007294,40.726639
3,4,2016/05/02 12:34:00,model_A,1,2076,Hawkins,-73.870964,40.773792,-73.989090,40.746052
4,5,2016/03/02 15:53:00,model_A,1,1289,Hawkins,-73.979012,40.745266,-73.979240,40.772480
...,...,...,...,...,...,...,...,...,...,...
8489,8490,2016/05/02 08:27:00,model_A,6,689,Hawkins,-73.992950,40.730621,-73.980537,40.751068
8490,8491,2016/06/05 07:56:00,model_B,3,538,Hawkins,-74.008171,40.738667,-73.994019,40.767262
8491,8492,2016/03/10 12:49:00,model_B,1,3093,Hawkins,-73.970123,40.765587,-73.789490,40.644848
8492,8493,2016/05/02 20:52:00,model_B,1,289,Hawkins,-73.978973,40.762123,-73.979095,40.772747


In [9]:
train = pd.merge(train_g1, train_w, on = 'id')
pd.set_option('display.max_columns', None)
train

,id,starting_time,model_van,num_packages,delivery_time,town,start_Longitude,start_Latitude,end_Longitude,end_Latitude,wea_conditions,wea_temp,wea_dptemp,wea_humidity,wea_windspeed,wea_winddir,wea_visibility,wea_pressure,wea_precipitation,wea_fog,wea_rain,wea_snow
0,1,2016/04/14 20:53:00,model_A,3,448,Hawkins,-73.950912,40.825974,-73.944740,40.814163,Clear,8.9,0.0,54.0,16.7,170,16.1,30.4,NaN,No,No,No
1,2,2016/05/30 23:12:00,model_B,1,784,Hawkins,-73.985680,40.760487,-73.942482,40.786232,Partly_Cloudy,22.8,18.9,79.0,9.3,0,16.1,30.0,NaN,No,No,No
2,3,2016/03/03 11:54:00,model_B,1,844,Hawkins,-73.985420,40.758389,-74.007294,40.726639,Clear,0.0,-15.6,31.0,9.3,250,16.1,30.2,NaN,No,No,No
3,4,2016/05/02 12:34:00,model_A,1,2076,Hawkins,-73.870964,40.773792,-73.989090,40.746052,Overcast,0.6,-1.7,85.0,13.0,300,4.0,30.1,NaN,No,No,No
4,5,2016/03/02 15:53:00,model_A,1,1289,Hawkins,-73.979012,40.745266,-73.979240,40.772480,Rain,13.9,12.2,89.0,11.1,170,1.6,30.0,5.1,No,Y,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8489,8490,2016/05/02 08:27:00,model_A,6,689,Hawkins,-73.992950,40.730621,-73.980537,40.751068,Snow,-0.6,-1.7,92.0,16.7,20,0.4,30.1,0.3,Y,No,Y
8490,8491,2016/06/05 07:56:00,model_B,3,538,Hawkins,-74.008171,40.738667,-73.994019,40.767262,Overcast,8.9,7.2,89.0,22.2,60,14.5,29.7,0.8,No,No,No
8491,8492,2016/03/10 12:49:00,model_B,1,3093,Hawkins,-73.970123,40.765587,-73.789490,40.644848,Mostly_Cloudy,20.6,13.3,63.0,7.4,50,14.5,30.1,NaN,No,No,No
8492,8493,2016/05/02 20:52:00,model_B,1,289,Hawkins,-73.978973,40.762123,-73.979095,40.772747,Clear,1.1,-10.6,42.0,18.5,310,16.1,30.3,NaN,No,No,No


In [10]:
# Merging Test Dataframes on id
test_g1 = pd.merge(test_m, test_g, on = 'id')
test_g1

,id,starting_time,model_van,num_packages,town,start_Longitude,start_Latitude,end_Longitude,end_Latitude
0,8495,2016/05/18 23:53:00,model_B,1,Hawkins,-74.001488,40.741226,-73.949318,40.780621
1,8496,2016/02/29 03:54:00,model_B,1,Hawkins,-73.973564,40.749073,-73.972549,40.760288
2,8497,2016/04/06 09:52:00,model_B,1,Hawkins,-73.975304,40.790001,-74.014374,40.714127
3,8498,2016/04/12 06:52:00,model_B,2,Hawkins,-74.001671,40.730682,-73.980858,40.760967
4,8499,2016/06/11 11:56:00,model_B,1,Hawkins,-73.973259,40.756893,-73.968254,40.767891
...,...,...,...,...,...,...,...,...,...
8491,16986,2016/05/27 09:53:00,model_A,1,Hawkins,-73.983032,40.742813,-73.988548,40.758240
8492,16987,2016/02/25 15:55:00,model_A,1,Hawkins,-73.962318,40.800091,-73.962067,40.804821
8493,16988,2016/01/23 04:16:00,model_A,2,Hawkins,-73.955788,40.772346,-74.001320,40.731155
8494,16989,2016/05/23 17:54:00,model_B,1,Hawkins,-73.983528,40.760189,-73.960625,40.769749


In [11]:
test = pd.merge(test_g1, test_w, on = 'id')
test

,id,starting_time,model_van,num_packages,town,start_Longitude,start_Latitude,end_Longitude,end_Latitude,wea_conditions,wea_temp,wea_dptemp,wea_humidity,wea_windspeed,wea_winddir,wea_visibility,wea_pressure,wea_precipitation,wea_fog,wea_rain,wea_snow
0,8495,2016/05/18 23:53:00,model_B,1,Hawkins,-74.001488,40.741226,-73.949318,40.780621,Overcast,12.8,6.1,64.0,0.0,0,16.1,30.1,0.0,No,No,No
1,8496,2016/02/29 03:54:00,model_B,1,Hawkins,-73.973564,40.749073,-73.972549,40.760288,Clear,10.0,-1.1,46.0,9.3,0,16.1,29.6,NaN,No,No,No
2,8497,2016/04/06 09:52:00,model_B,1,Hawkins,-73.975304,40.790001,-74.014374,40.714127,Haze,23.3,17.8,71.0,9.3,250,8.0,30.0,NaN,No,No,No
3,8498,2016/04/12 06:52:00,model_B,2,Hawkins,-74.001671,40.730682,-73.980858,40.760967,Scattered_Clouds,3.9,-6.1,49.0,18.5,300,16.1,30.2,NaN,No,No,No
4,8499,2016/06/11 11:56:00,model_B,1,Hawkins,-73.973259,40.756893,-73.968254,40.767891,Clear,13.9,0.0,39.0,11.1,320,16.1,30.2,NaN,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8491,16986,2016/05/27 09:53:00,model_A,1,Hawkins,-73.983032,40.742813,-73.988548,40.758240,Haze,26.1,16.7,56.0,5.6,0,9.7,30.1,NaN,No,No,No
8492,16987,2016/02/25 15:55:00,model_A,1,Hawkins,-73.962318,40.800091,-73.962067,40.804821,Light_Rain,7.2,-1.1,56.0,20.4,270,16.1,29.3,0.0,No,Y,No
8493,16988,2016/01/23 04:16:00,model_A,2,Hawkins,-73.955788,40.772346,-74.001320,40.731155,Heavy_Snow,-4.4,-7.2,81.0,18.5,50,0.4,29.9,0.8,No,No,Y
8494,16989,2016/05/23 17:54:00,model_B,1,Hawkins,-73.983528,40.760189,-73.960625,40.769749,Light_Rain,17.8,11.7,67.0,9.3,190,8.0,29.9,0.3,No,Y,No


In [12]:
# EDA

In [13]:
train.isnull().sum()

id                      0
starting_time           0
model_van               0
num_packages            0
delivery_time           0
town                    0
start_Longitude         0
start_Latitude          0
end_Longitude           0
end_Latitude            0
wea_conditions          0
wea_temp                2
wea_dptemp              2
wea_humidity            2
wea_windspeed         657
wea_winddir             0
wea_visibility        229
wea_pressure          226
wea_precipitation    7213
wea_fog                 0
wea_rain                0
wea_snow                0
dtype: int64

In [14]:
test.isnull().sum()

id                      0
starting_time           0
model_van               0
num_packages            0
town                    0
start_Longitude         0
start_Latitude          0
end_Longitude           0
end_Latitude            0
wea_conditions          0
wea_temp                9
wea_dptemp              9
wea_humidity            9
wea_windspeed         629
wea_winddir             0
wea_visibility        230
wea_pressure          226
wea_precipitation    7186
wea_fog                 0
wea_rain                0
wea_snow                0
dtype: int64

In [15]:
train.dtypes

id                     int64
starting_time         object
model_van             object
num_packages           int64
delivery_time          int64
town                  object
start_Longitude      float64
start_Latitude       float64
end_Longitude        float64
end_Latitude         float64
wea_conditions        object
wea_temp             float64
wea_dptemp           float64
wea_humidity         float64
wea_windspeed        float64
wea_winddir            int64
wea_visibility       float64
wea_pressure         float64
wea_precipitation    float64
wea_fog               object
wea_rain              object
wea_snow              object
dtype: object

In [16]:
train.nunique()

id                   8494
starting_time        6828
model_van               2
num_packages            6
delivery_time        1945
town                    1
start_Longitude      5368
start_Latitude       6812
end_Longitude        5547
end_Latitude         7013
wea_conditions         13
wea_temp              125
wea_dptemp            126
wea_humidity           88
wea_windspeed          20
wea_winddir            37
wea_visibility         17
wea_pressure           15
wea_precipitation      36
wea_fog                 2
wea_rain                2
wea_snow                2
dtype: int64

In [17]:
train['wea_fog'].unique()

array(['No', 'Y'], dtype=object)

In [18]:
train['wea_rain'].unique()

array(['No', 'Y'], dtype=object)

In [19]:
train['wea_snow'].unique()

array(['No', 'Y'], dtype=object)

In [20]:
train['town'].unique()

array(['Hawkins'], dtype=object)

In [21]:
# Concatenaning Train and Test to do feature engineering

In [22]:
train.columns

Index(['id', 'starting_time', 'model_van', 'num_packages', 'delivery_time',
       'town', 'start_Longitude', 'start_Latitude', 'end_Longitude',
       'end_Latitude', 'wea_conditions', 'wea_temp', 'wea_dptemp',
       'wea_humidity', 'wea_windspeed', 'wea_winddir', 'wea_visibility',
       'wea_pressure', 'wea_precipitation', 'wea_fog', 'wea_rain', 'wea_snow'],
      dtype='object')

In [23]:
test.columns

Index(['id', 'starting_time', 'model_van', 'num_packages', 'town',
       'start_Longitude', 'start_Latitude', 'end_Longitude', 'end_Latitude',
       'wea_conditions', 'wea_temp', 'wea_dptemp', 'wea_humidity',
       'wea_windspeed', 'wea_winddir', 'wea_visibility', 'wea_pressure',
       'wea_precipitation', 'wea_fog', 'wea_rain', 'wea_snow'],
      dtype='object')

In [24]:
train['delivery_time'].unique()

array([ 448,  784,  844, ..., 1130, 1685, 3093])

In [25]:
train['delivery_time'].min()

60

In [26]:
train['delivery_time'].max()

3300

In [27]:
test['delivery_time'] = 0

In [28]:
test

,id,starting_time,model_van,num_packages,town,start_Longitude,start_Latitude,end_Longitude,end_Latitude,wea_conditions,wea_temp,wea_dptemp,wea_humidity,wea_windspeed,wea_winddir,wea_visibility,wea_pressure,wea_precipitation,wea_fog,wea_rain,wea_snow,delivery_time
0,8495,2016/05/18 23:53:00,model_B,1,Hawkins,-74.001488,40.741226,-73.949318,40.780621,Overcast,12.8,6.1,64.0,0.0,0,16.1,30.1,0.0,No,No,No,0
1,8496,2016/02/29 03:54:00,model_B,1,Hawkins,-73.973564,40.749073,-73.972549,40.760288,Clear,10.0,-1.1,46.0,9.3,0,16.1,29.6,NaN,No,No,No,0
2,8497,2016/04/06 09:52:00,model_B,1,Hawkins,-73.975304,40.790001,-74.014374,40.714127,Haze,23.3,17.8,71.0,9.3,250,8.0,30.0,NaN,No,No,No,0
3,8498,2016/04/12 06:52:00,model_B,2,Hawkins,-74.001671,40.730682,-73.980858,40.760967,Scattered_Clouds,3.9,-6.1,49.0,18.5,300,16.1,30.2,NaN,No,No,No,0
4,8499,2016/06/11 11:56:00,model_B,1,Hawkins,-73.973259,40.756893,-73.968254,40.767891,Clear,13.9,0.0,39.0,11.1,320,16.1,30.2,NaN,No,No,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8491,16986,2016/05/27 09:53:00,model_A,1,Hawkins,-73.983032,40.742813,-73.988548,40.758240,Haze,26.1,16.7,56.0,5.6,0,9.7,30.1,NaN,No,No,No,0
8492,16987,2016/02/25 15:55:00,model_A,1,Hawkins,-73.962318,40.800091,-73.962067,40.804821,Light_Rain,7.2,-1.1,56.0,20.4,270,16.1,29.3,0.0,No,Y,No,0
8493,16988,2016/01/23 04:16:00,model_A,2,Hawkins,-73.955788,40.772346,-74.001320,40.731155,Heavy_Snow,-4.4,-7.2,81.0,18.5,50,0.4,29.9,0.8,No,No,Y,0
8494,16989,2016/05/23 17:54:00,model_B,1,Hawkins,-73.983528,40.760189,-73.960625,40.769749,Light_Rain,17.8,11.7,67.0,9.3,190,8.0,29.9,0.3,No,Y,No,0


In [29]:
dataset = train.append(test)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [30]:
dataset

,delivery_time,end_Latitude,end_Longitude,id,model_van,num_packages,start_Latitude,start_Longitude,starting_time,town,wea_conditions,wea_dptemp,wea_fog,wea_humidity,wea_precipitation,wea_pressure,wea_rain,wea_snow,wea_temp,wea_visibility,wea_winddir,wea_windspeed
0,448,40.814163,-73.944740,1,model_A,3,40.825974,-73.950912,2016/04/14 20:53:00,Hawkins,Clear,0.0,No,54.0,NaN,30.4,No,No,8.9,16.1,170,16.7
1,784,40.786232,-73.942482,2,model_B,1,40.760487,-73.985680,2016/05/30 23:12:00,Hawkins,Partly_Cloudy,18.9,No,79.0,NaN,30.0,No,No,22.8,16.1,0,9.3
2,844,40.726639,-74.007294,3,model_B,1,40.758389,-73.985420,2016/03/03 11:54:00,Hawkins,Clear,-15.6,No,31.0,NaN,30.2,No,No,0.0,16.1,250,9.3
3,2076,40.746052,-73.989090,4,model_A,1,40.773792,-73.870964,2016/05/02 12:34:00,Hawkins,Overcast,-1.7,No,85.0,NaN,30.1,No,No,0.6,4.0,300,13.0
4,1289,40.772480,-73.979240,5,model_A,1,40.745266,-73.979012,2016/03/02 15:53:00,Hawkins,Rain,12.2,No,89.0,5.1,30.0,Y,No,13.9,1.6,170,11.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8491,0,40.758240,-73.988548,16986,model_A,1,40.742813,-73.983032,2016/05/27 09:53:00,Hawkins,Haze,16.7,No,56.0,NaN,30.1,No,No,26.1,9.7,0,5.6
8492,0,40.804821,-73.962067,16987,model_A,1,40.800091,-73.962318,2016/02/25 15:55:00,Hawkins,Light_Rain,-1.1,No,56.0,0.0,29.3,Y,No,7.2,16.1,270,20.4
8493,0,40.731155,-74.001320,16988,model_A,2,40.772346,-73.955788,2016/01/23 04:16:00,Hawkins,Heavy_Snow,-7.2,No,81.0,0.8,29.9,No,Y,-4.4,0.4,50,18.5
8494,0,40.769749,-73.960625,16989,model_B,1,40.760189,-73.983528,2016/05/23 17:54:00,Hawkins,Light_Rain,11.7,No,67.0,0.3,29.9,Y,No,17.8,8.0,190,9.3


In [31]:
dataset

,delivery_time,end_Latitude,end_Longitude,id,model_van,num_packages,start_Latitude,start_Longitude,starting_time,town,wea_conditions,wea_dptemp,wea_fog,wea_humidity,wea_precipitation,wea_pressure,wea_rain,wea_snow,wea_temp,wea_visibility,wea_winddir,wea_windspeed
0,448,40.814163,-73.944740,1,model_A,3,40.825974,-73.950912,2016/04/14 20:53:00,Hawkins,Clear,0.0,No,54.0,NaN,30.4,No,No,8.9,16.1,170,16.7
1,784,40.786232,-73.942482,2,model_B,1,40.760487,-73.985680,2016/05/30 23:12:00,Hawkins,Partly_Cloudy,18.9,No,79.0,NaN,30.0,No,No,22.8,16.1,0,9.3
2,844,40.726639,-74.007294,3,model_B,1,40.758389,-73.985420,2016/03/03 11:54:00,Hawkins,Clear,-15.6,No,31.0,NaN,30.2,No,No,0.0,16.1,250,9.3
3,2076,40.746052,-73.989090,4,model_A,1,40.773792,-73.870964,2016/05/02 12:34:00,Hawkins,Overcast,-1.7,No,85.0,NaN,30.1,No,No,0.6,4.0,300,13.0
4,1289,40.772480,-73.979240,5,model_A,1,40.745266,-73.979012,2016/03/02 15:53:00,Hawkins,Rain,12.2,No,89.0,5.1,30.0,Y,No,13.9,1.6,170,11.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8491,0,40.758240,-73.988548,16986,model_A,1,40.742813,-73.983032,2016/05/27 09:53:00,Hawkins,Haze,16.7,No,56.0,NaN,30.1,No,No,26.1,9.7,0,5.6
8492,0,40.804821,-73.962067,16987,model_A,1,40.800091,-73.962318,2016/02/25 15:55:00,Hawkins,Light_Rain,-1.1,No,56.0,0.0,29.3,Y,No,7.2,16.1,270,20.4
8493,0,40.731155,-74.001320,16988,model_A,2,40.772346,-73.955788,2016/01/23 04:16:00,Hawkins,Heavy_Snow,-7.2,No,81.0,0.8,29.9,No,Y,-4.4,0.4,50,18.5
8494,0,40.769749,-73.960625,16989,model_B,1,40.760189,-73.983528,2016/05/23 17:54:00,Hawkins,Light_Rain,11.7,No,67.0,0.3,29.9,Y,No,17.8,8.0,190,9.3


In [32]:
# Splitting Date in Month, Day and Seconds of the day
dataset['month'] = pd.DatetimeIndex(dataset['starting_time']).month
dataset['day'] = pd.DatetimeIndex(dataset['starting_time']).day
dataset['seconds'] = ( pd.DatetimeIndex(dataset['starting_time']).hour * 60) +  (pd.DatetimeIndex(dataset['starting_time']).minute * 60)

In [33]:
dataset.drop('starting_time', axis=1, inplace=True)

In [34]:
dataset

,delivery_time,end_Latitude,end_Longitude,id,model_van,num_packages,start_Latitude,start_Longitude,town,wea_conditions,wea_dptemp,wea_fog,wea_humidity,wea_precipitation,wea_pressure,wea_rain,wea_snow,wea_temp,wea_visibility,wea_winddir,wea_windspeed,month,day,seconds
0,448,40.814163,-73.944740,1,model_A,3,40.825974,-73.950912,Hawkins,Clear,0.0,No,54.0,NaN,30.4,No,No,8.9,16.1,170,16.7,4,14,4380
1,784,40.786232,-73.942482,2,model_B,1,40.760487,-73.985680,Hawkins,Partly_Cloudy,18.9,No,79.0,NaN,30.0,No,No,22.8,16.1,0,9.3,5,30,2100
2,844,40.726639,-74.007294,3,model_B,1,40.758389,-73.985420,Hawkins,Clear,-15.6,No,31.0,NaN,30.2,No,No,0.0,16.1,250,9.3,3,3,3900
3,2076,40.746052,-73.989090,4,model_A,1,40.773792,-73.870964,Hawkins,Overcast,-1.7,No,85.0,NaN,30.1,No,No,0.6,4.0,300,13.0,5,2,2760
4,1289,40.772480,-73.979240,5,model_A,1,40.745266,-73.979012,Hawkins,Rain,12.2,No,89.0,5.1,30.0,Y,No,13.9,1.6,170,11.1,3,2,4080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8491,0,40.758240,-73.988548,16986,model_A,1,40.742813,-73.983032,Hawkins,Haze,16.7,No,56.0,NaN,30.1,No,No,26.1,9.7,0,5.6,5,27,3720
8492,0,40.804821,-73.962067,16987,model_A,1,40.800091,-73.962318,Hawkins,Light_Rain,-1.1,No,56.0,0.0,29.3,Y,No,7.2,16.1,270,20.4,2,25,4200
8493,0,40.731155,-74.001320,16988,model_A,2,40.772346,-73.955788,Hawkins,Heavy_Snow,-7.2,No,81.0,0.8,29.9,No,Y,-4.4,0.4,50,18.5,1,23,1200
8494,0,40.769749,-73.960625,16989,model_B,1,40.760189,-73.983528,Hawkins,Light_Rain,11.7,No,67.0,0.3,29.9,Y,No,17.8,8.0,190,9.3,5,23,4260


In [35]:
# Handling Null Values
dataset.isnull().sum()

delivery_time            0
end_Latitude             0
end_Longitude            0
id                       0
model_van                0
num_packages             0
start_Latitude           0
start_Longitude          0
town                     0
wea_conditions           0
wea_dptemp              11
wea_fog                  0
wea_humidity            11
wea_precipitation    14399
wea_pressure           452
wea_rain                 0
wea_snow                 0
wea_temp                11
wea_visibility         459
wea_winddir              0
wea_windspeed         1286
month                    0
day                      0
seconds                  0
dtype: int64

In [36]:
# Precipitation has too many Null values
dataset.drop('wea_precipitation', axis=1, inplace=True)

In [37]:
dataset.isnull().sum()

delivery_time         0
end_Latitude          0
end_Longitude         0
id                    0
model_van             0
num_packages          0
start_Latitude        0
start_Longitude       0
town                  0
wea_conditions        0
wea_dptemp           11
wea_fog               0
wea_humidity         11
wea_pressure        452
wea_rain              0
wea_snow              0
wea_temp             11
wea_visibility      459
wea_winddir           0
wea_windspeed      1286
month                 0
day                   0
seconds               0
dtype: int64

In [38]:
dataset.mean()

delivery_time       384.663096
end_Latitude         40.753171
end_Longitude       -73.974243
id                 8495.500000
num_packages          1.676280
start_Latitude       40.752094
start_Longitude     -73.975875
wea_dptemp            2.488403
wea_humidity         57.594264
wea_pressure         30.009765
wea_temp             11.522881
wea_visibility       13.761829
wea_winddir          82.728664
wea_windspeed         9.771600
month                 3.446321
day                  15.130253
seconds            3785.904650
dtype: float64

In [39]:
dataset.fillna(dataset.mean(), inplace=True)

In [40]:
dataset.isnull().sum()

delivery_time      0
end_Latitude       0
end_Longitude      0
id                 0
model_van          0
num_packages       0
start_Latitude     0
start_Longitude    0
town               0
wea_conditions     0
wea_dptemp         0
wea_fog            0
wea_humidity       0
wea_pressure       0
wea_rain           0
wea_snow           0
wea_temp           0
wea_visibility     0
wea_winddir        0
wea_windspeed      0
month              0
day                0
seconds            0
dtype: int64

In [41]:
dataset.dtypes

delivery_time        int64
end_Latitude       float64
end_Longitude      float64
id                   int64
model_van           object
num_packages         int64
start_Latitude     float64
start_Longitude    float64
town                object
wea_conditions      object
wea_dptemp         float64
wea_fog             object
wea_humidity       float64
wea_pressure       float64
wea_rain            object
wea_snow            object
wea_temp           float64
wea_visibility     float64
wea_winddir          int64
wea_windspeed      float64
month                int64
day                  int64
seconds              int64
dtype: object

In [42]:
# Holidays

In [43]:
dataset['holidays'] = 0

In [44]:
holidays['month'] = pd.DatetimeIndex(holidays['date']).month
holidays['day'] = pd.DatetimeIndex(holidays['date']).day

In [45]:
dataset['monthday'] = dataset.month.astype(str) + dataset.day.astype(str)
holidays['monthday'] = holidays.month.astype(str) + holidays.day.astype(str)

In [46]:
dataset['monthday'] = dataset['monthday'].astype(int)

In [47]:
dataset

,delivery_time,end_Latitude,end_Longitude,id,model_van,num_packages,start_Latitude,start_Longitude,town,wea_conditions,wea_dptemp,wea_fog,wea_humidity,wea_pressure,wea_rain,wea_snow,wea_temp,wea_visibility,wea_winddir,wea_windspeed,month,day,seconds,holidays,monthday
0,448,40.814163,-73.944740,1,model_A,3,40.825974,-73.950912,Hawkins,Clear,0.0,No,54.0,30.4,No,No,8.9,16.1,170,16.7,4,14,4380,0,414
1,784,40.786232,-73.942482,2,model_B,1,40.760487,-73.985680,Hawkins,Partly_Cloudy,18.9,No,79.0,30.0,No,No,22.8,16.1,0,9.3,5,30,2100,0,530
2,844,40.726639,-74.007294,3,model_B,1,40.758389,-73.985420,Hawkins,Clear,-15.6,No,31.0,30.2,No,No,0.0,16.1,250,9.3,3,3,3900,0,33
3,2076,40.746052,-73.989090,4,model_A,1,40.773792,-73.870964,Hawkins,Overcast,-1.7,No,85.0,30.1,No,No,0.6,4.0,300,13.0,5,2,2760,0,52
4,1289,40.772480,-73.979240,5,model_A,1,40.745266,-73.979012,Hawkins,Rain,12.2,No,89.0,30.0,Y,No,13.9,1.6,170,11.1,3,2,4080,0,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8491,0,40.758240,-73.988548,16986,model_A,1,40.742813,-73.983032,Hawkins,Haze,16.7,No,56.0,30.1,No,No,26.1,9.7,0,5.6,5,27,3720,0,527
8492,0,40.804821,-73.962067,16987,model_A,1,40.800091,-73.962318,Hawkins,Light_Rain,-1.1,No,56.0,29.3,Y,No,7.2,16.1,270,20.4,2,25,4200,0,225
8493,0,40.731155,-74.001320,16988,model_A,2,40.772346,-73.955788,Hawkins,Heavy_Snow,-7.2,No,81.0,29.9,No,Y,-4.4,0.4,50,18.5,1,23,1200,0,123
8494,0,40.769749,-73.960625,16989,model_B,1,40.760189,-73.983528,Hawkins,Light_Rain,11.7,No,67.0,29.9,Y,No,17.8,8.0,190,9.3,5,23,4260,0,523


In [48]:
holidays

,date,holiday,month,day,monthday
0,2016-01-01,Y,1,1,11
1,2016-01-18,Y,1,18,118
2,2016-02-12,Y,2,12,212
3,2016-02-15,Y,2,15,215
4,2016-05-08,Y,5,8,58
5,2016-05-30,Y,5,30,530
6,2016-06-19,Y,6,19,619
7,2016-07-04,Y,7,4,74
8,2016-09-05,Y,9,5,95
9,2016-10-10,Y,10,10,1010


In [49]:
dataset.loc[dataset.monthday == 11, 'holidays'] = 1
dataset.loc[dataset.monthday == 118, 'holidays'] = 1
dataset.loc[dataset.monthday == 212, 'holidays'] = 1
dataset.loc[dataset.monthday == 215, 'holidays'] = 1
dataset.loc[dataset.monthday == 58, 'holidays'] = 1
dataset.loc[dataset.monthday == 530, 'holidays'] = 1
dataset.loc[dataset.monthday == 619, 'holidays'] = 1
dataset.loc[dataset.monthday == 74, 'holidays'] = 1
dataset.loc[dataset.monthday == 95, 'holidays'] = 1
dataset.loc[dataset.monthday == 1010, 'holidays'] = 1
dataset.loc[dataset.monthday == 1111, 'holidays'] = 1
dataset.loc[dataset.monthday == 1124, 'holidays'] = 1
dataset.loc[dataset.monthday == 1226, 'holidays'] = 1

In [50]:
dataset.drop('monthday', axis=1, inplace=True)

In [51]:
def label_encoder(df):
    def numerical_features(df):
        columns = df.columns
        return df._get_numeric_data().columns

    def categorical_features(df):
        numerical_columns = numerical_features(df)
        return(list(set(df.columns) - set(numerical_columns)))
    
    categorical = categorical_features(df)
    # Creating the label encoder object
    le =  LabelEncoder()
    
    # Iterating over the "object" variables to transform the categories into numbers 
    for col in categorical:
        df[col] = le.fit_transform(df[col].astype(str))
    return df

In [52]:
final_dataset = label_encoder(dataset)

In [53]:
final_dataset.dtypes

delivery_time        int64
end_Latitude       float64
end_Longitude      float64
id                   int64
model_van            int64
num_packages         int64
start_Latitude     float64
start_Longitude    float64
town                 int64
wea_conditions       int64
wea_dptemp         float64
wea_fog              int64
wea_humidity       float64
wea_pressure       float64
wea_rain             int64
wea_snow             int64
wea_temp           float64
wea_visibility     float64
wea_winddir          int64
wea_windspeed      float64
month                int64
day                  int64
seconds              int64
holidays             int64
dtype: object

In [54]:
final_dataset

,delivery_time,end_Latitude,end_Longitude,id,model_van,num_packages,start_Latitude,start_Longitude,town,wea_conditions,wea_dptemp,wea_fog,wea_humidity,wea_pressure,wea_rain,wea_snow,wea_temp,wea_visibility,wea_winddir,wea_windspeed,month,day,seconds,holidays
0,448,40.814163,-73.944740,1,0,3,40.825974,-73.950912,0,0,0.0,0,54.0,30.4,0,0,8.9,16.1,170,16.7,4,14,4380,0
1,784,40.786232,-73.942482,2,1,1,40.760487,-73.985680,0,9,18.9,0,79.0,30.0,0,0,22.8,16.1,0,9.3,5,30,2100,1
2,844,40.726639,-74.007294,3,1,1,40.758389,-73.985420,0,0,-15.6,0,31.0,30.2,0,0,0.0,16.1,250,9.3,3,3,3900,0
3,2076,40.746052,-73.989090,4,0,1,40.773792,-73.870964,0,8,-1.7,0,85.0,30.1,0,0,0.6,4.0,300,13.0,5,2,2760,0
4,1289,40.772480,-73.979240,5,0,1,40.745266,-73.979012,0,10,12.2,0,89.0,30.0,1,0,13.9,1.6,170,11.1,3,2,4080,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8491,0,40.758240,-73.988548,16986,0,1,40.742813,-73.983032,0,1,16.7,0,56.0,30.1,0,0,26.1,9.7,0,5.6,5,27,3720,0
8492,0,40.804821,-73.962067,16987,0,1,40.800091,-73.962318,0,5,-1.1,0,56.0,29.3,1,0,7.2,16.1,270,20.4,2,25,4200,0
8493,0,40.731155,-74.001320,16988,0,2,40.772346,-73.955788,0,3,-7.2,0,81.0,29.9,0,1,-4.4,0.4,50,18.5,1,23,1200,0
8494,0,40.769749,-73.960625,16989,1,1,40.760189,-73.983528,0,5,11.7,0,67.0,29.9,1,0,17.8,8.0,190,9.3,5,23,4260,0


In [55]:
# Splitting the Dataset again
X = final_dataset[final_dataset['delivery_time'] != 0].reset_index(drop=True)
y = X['delivery_time']
X_test = final_dataset[final_dataset['delivery_time'] == 0].reset_index(drop=True)

In [56]:
X.drop('delivery_time', axis=1, inplace=True)
X_test.drop('delivery_time', axis=1, inplace=True)

In [57]:
# MODELLING

In [58]:
x_train, x_val, y_train, y_val = train_test_split(X,y,test_size=0.1,train_size=0.9, random_state = 0)

In [59]:
# regressor = RandomForestRegressor()
# parameters = [{'n_estimators' : [200,250,300], 'max_features' : ['auto','sqrt','log2']}]
# grid_search = GridSearchCV(estimator = regressor, param_grid = parameters)
# grid_search = grid_search.fit(x_train, y_train)
# best_parameters = grid_search.best_params_
# best_accuracy = grid_search.best_score_

In [60]:
# best_parameters

In [61]:
regr = RandomForestRegressor(n_estimators = 250, max_features = 'auto') 
regr.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=250, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [62]:
pred_val = regr.predict(x_val)

In [63]:
mean_absolute_error(y_val, pred_val)

225.39834823529415

In [64]:
regr.fit(X, y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=250, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [65]:
pred = regr.predict(X_test)

In [66]:
sample_submission['delivery_time']= pred
sample_submission.to_csv('my_submission_03.csv',index=False)